In [ ]:
import pandas as pd, numpy as np, tensorflow as tf,  keras_nlp as nlp, keras
from sklearn.model_selection import train_test_split

In [1]:
#Generate split
generate_split = False
dev = True
if generate_split:
    df = pd.read_csv('./data/en-fr.csv', encoding='latin-1')
    train, test = train_test_split(df, test_size=0.2)
    train, test = pd.DataFrame(train), pd.DataFrame(test) #My LSP is bugged, this shut up the warnings
    train.to_csv('./data/train.csv', index=False)
    train.iloc[:10000].to_csv('./data/smol_train.csv', index=False)
    test.to_csv('./data/test.csv', index=False)
    test.iloc[:1000].to_csv('./data/smol_test.csv', index=False)
    if dev:
        train = train.iloc[:10000]
        test = test.iloc[:1000]
else:
    if dev:
        train = pd.read_csv('./data/smol_train.csv', encoding='latin-1')
        test = pd.read_csv('./data/smol_test.csv', encoding='latin-1')
    else:
        train = pd.read_csv('./data/train.csv', encoding='latin-1')
        test = pd.read_csv('./data/test.csv', encoding='latin-1')
print(train.shape, test.shape)
print(train.head())
print(test.head())

(10000, 2) (1000, 2)
                                                  en                                                 fr
0  This is accomplished through a range of activi...  Toute une gamme d'activitÃÂ©s lui permet d'at...
1  Ms. Sabir was understandably very offended by ...  Mme Sabir a tout naturellement ÃÂ©tÃÂ© offen...
2  She will not report the interest income from h...  Elle n'a pas ÃÂ  inclure dans sa dÃÂ©clarati...
3  On the part of Mr. Morken, while he may first ...  En ce qui concerne M. Morken, mÃÂªme si, au d...
4  E Jean-Pierre Kingsley Chief Electoral Officer...  ÃÂ Jean-Pierre Kingsley Directeur gÃÂ©nÃÂ©...
                                                  en                                                 fr
0  FinTRAC will also have primary responsibility ...  Le Centre aura aussi la responsabilitÃÂ© de s...
1   It is required to report to Parliament annually.  Maintien de l'autonomie financiÃÂ¨re dans ses...
2  The bilingualism of the individuals also

---------------------------------------------------------------------------
NameError                                 Traceback (most recent call last)
Cell In[3], line 1
----> 1 train

NameError: name 'train' is not defined


In [ ]:
# Examples from kerasnlp
# WordPiece Tokenizer
ENG_VOCAB_SIZE = 30000
FR_VOCAB_SIZE = 30000

# Preprocessing params.
PRETRAINING_BATCH_SIZE = 128
FINETUNING_BATCH_SIZE = 32
SEQ_LENGTH = 128
MASK_RATE = 0.25
PREDICTIONS_PER_SEQ = 32

# Model params.
NUM_LAYERS = 3
MODEL_DIM = 256
#INTERMEDIATE_DIM = 512
#NUM_HEADS = 4
DROPOUT = 0.1
NORM_EPSILON = 1e-5

EMBED_DIM = 256
INTERMEDIATE_DIM = 2048
NUM_HEADS = 8
MAX_SEQUENCE_LENGTH = 64

# Training params.
PRETRAINING_LEARNING_RATE = 5e-4
PRETRAINING_EPOCHS = 8
FINETUNING_LEARNING_RATE = 5e-5
FINETUNING_EPOCHS = 3

In [ ]:
trainfile = './data/smol_train.csv' if dev else './data/train.csv'
testfile = './data/smol_test.csv' if dev else './data/test.csv'
train_data = tf.data.experimental.CsvDataset(trainfile, [tf.string, tf.string], header=True).batch(FINETUNING_BATCH_SIZE)
test_data = tf.data.experimental.CsvDataset(trainfile, [tf.string, tf.string], header=True).batch(FINETUNING_BATCH_SIZE)

In [3]:
print(train_data.unbatch().batch(4).take(1).get_single_element())

(<tf.Tensor: shape=(4,), dtype=string, numpy=
array([b"This is accomplished through a range of activities, from the monitoring of other countries' disease status to the control of importations, including the certification of import documentation, border inspections, testing and quarantine.",
       b'Ms. Sabir was understandably very offended by the use of racial epithets in the workplace, and by what she perceived to be the unfair treatment of Gloria Baptiste.',
       b'She will not report the interest income from her bank account in England on her Canadian return.',
       b'On the part of Mr. Morken, while he may first have considered that he did not wish to provide a reference, he did so when it became important to the timely completion of the process.'],
      dtype=object)>, <tf.Tensor: shape=(4,), dtype=string, numpy=
array([b"Toute une gamme d'activit\xc3\x83\xc2\xa9s lui permet d'atteindre cet objectif : la surveillance de la situation d'autres pays relativement aux maladies,

In [ ]:
def train_word_piece(text_samples, vocab_size, reserved_tokens):
    word_piece_ds = tf.data.Dataset.from_tensor_slices(text_samples)
    vocab = nlp.tokenizers.compute_word_piece_vocabulary(
        word_piece_ds.batch(1000).prefetch(2),
        vocabulary_size=vocab_size,
        reserved_tokens=reserved_tokens,
    )
    return vocab

reserved_tokens = ["[PAD]", "[UNK]", "[START]", "[END]"]

eng_vocab = train_word_piece(train['en'], ENG_VOCAB_SIZE, reserved_tokens)
fr_vocab = train_word_piece(train['fr'], FR_VOCAB_SIZE, reserved_tokens)

In [4]:
print(eng_vocab)
print(fr_vocab)

['[PAD]', '[UNK]', '[START]', '[END]', '!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '<', '=', '>', '?', '@', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', '\\', ']', '_', '`', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '{', '|', '}', '~', '\xa0', '¡', '¢', '£', '¤', '¥', '¦', '§', '¨', '©', 'ª', '«', '¬', '®', '¯', '°', '±', '²', '³', '´', 'µ', '¶', '·', '¸', '¹', 'º', '»', '¼', '½', '¾', 'Â', 'Ã', 'the', 'of', 'and', 'to', 'in', '##s', 'for', 'that', 'is', 'The', 'on', 'Ã¢Â', 'or', 'be', 'by', 'with', 'as', 'are', '##ing', '##Ã¢Â', 'from', 'an', 'Canada', 'at', 'not', '##ed', 'was', 'Â¢', 'this', 'will', 'have', '##d', 'which', 'it', '##ly', 'has', 'their', 'other', 'its', 'In', 'were', '##t', 'Canadian', 'information', 'al

In [ ]:
eng_tokenizer = nlp.tokenizers.WordPieceTokenizer(eng_vocab)
fr_tokenizer = nlp.tokenizers.WordPieceTokenizer(fr_vocab)

In [ ]:
def create_model():

    # Encoder
    encoder_inputs = keras.Input(shape=(None,), name="encoder_inputs")
    
    x = nlp.layers.TokenAndPositionEmbedding(
        vocabulary_size=ENG_VOCAB_SIZE,
        sequence_length=MAX_SEQUENCE_LENGTH,
        embedding_dim=EMBED_DIM,
    )(encoder_inputs)
    
    encoder_outputs = nlp.layers.TransformerEncoder(
        intermediate_dim=INTERMEDIATE_DIM, num_heads=NUM_HEADS
    )(inputs=x)
    encoder = keras.Model(encoder_inputs, encoder_outputs)
    
    
    # Decoder
    decoder_inputs = keras.Input(shape=(None,), name="decoder_inputs")
    encoded_seq_inputs = keras.Input(shape=(None, EMBED_DIM), name="decoder_state_inputs")
    
    x = nlp.layers.TokenAndPositionEmbedding(
        vocabulary_size=FR_VOCAB_SIZE,
        sequence_length=MAX_SEQUENCE_LENGTH,
        embedding_dim=EMBED_DIM,
    )(decoder_inputs)
    
    x = nlp.layers.TransformerDecoder(
        intermediate_dim=INTERMEDIATE_DIM, num_heads=NUM_HEADS
    )(decoder_sequence=x, encoder_sequence=encoded_seq_inputs)
    x = keras.layers.Dropout(0.5)(x)
    decoder_outputs = keras.layers.Dense(FR_VOCAB_SIZE, activation="softmax")(x)
    decoder = keras.Model(
        [
            decoder_inputs,
            encoded_seq_inputs,
        ],
        decoder_outputs,
    )
    decoder_outputs = decoder([decoder_inputs, encoder_outputs])
    
    transformer = keras.Model(
        [encoder_inputs, decoder_inputs],
        decoder_outputs,
        name="transformer",
    )
    return transformer
    

In [5]:
transformer = create_model()
transformer.summary()

Model: "transformer"
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ encoder_inputs (InputLayer)   │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_and_position_embedding… │ (None, None, 256)         │       7,696,384 │ encoder_inputs[0][0]       │
│ (TokenAndPositionEmbedding)   │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ decoder_inputs (InputLayer)   │ (None, None)              │               0 │ -                          

In [6]:
transformer.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
transformer.fit(train_data, epochs=10, validation_data=test_data)

Epoch 1/10
---------------------------------------------------------------------------
ValueError                                Traceback (most recent call last)
Cell In[43], line 2
      1 transformer.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
----> 2 transformer.fit(train_data, epochs=10, validation_data=test_data)

File ~/anaconda3/envs/ketsu/lib/python3.9/site-packages/keras/src/utils/traceback_utils.py:122, in filter_traceback.<locals>.error_handler(*args, **kwargs)
    119     filtered_tb = _process_traceback_frames(e.__traceback__)
    120     # To get the full stack trace, call:
    121     # `keras.config.disable_traceback_filtering()`
--> 122     raise e.with_traceback(filtered_tb) from None
    123 finally:
    124     del filtered_tb

File ~/anaconda3/envs/ketsu/lib/python3.9/site-packages/keras/src/layers/input_spec.py:156, in assert_input_compatibility(input_spec, inputs, layer_name)
    154 inputs = tree.flatten(inputs)
    155 if l